In [1]:
import cv2
import numpy as np
import math

In [2]:
import autopy

In [3]:
import mediapipe as mp

In [4]:
cap = cv2.VideoCapture(0)

In [5]:
wCam = 720
hCam = 720

cap.set(3,wCam)
cap.set(4,hCam)

True

In [6]:
wScrn , hScrn = autopy.screen.size()

In [7]:
mpHands = mp.solutions.hands
mpDraw = mp.solutions.drawing_utils

In [8]:
hands = mpHands.Hands(
                       static_image_mode=False,  
                       max_num_hands=2,
                       min_detection_confidence = 0.7 ,
                       min_tracking_confidence = 0.5 
                     )

In [9]:
tipIds = [4, 8, 12, 16, 20]

In [10]:
def findHands(img ,draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            if draw:
                mpDraw.draw_landmarks(img, handLms,mpHands.HAND_CONNECTIONS)
    
    return img,results

In [11]:
def findPosition(img ,results,draw=True):
    xList = []
    yList = []
    bbox = []
    
    lmList = []
    
    h, w = img.shape[:2]
    
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[0]
        
        for id, lm in enumerate(myHand.landmark):
            cx, cy = int(lm.x*w), int(lm.y*h)

            xList.append(cx)
            yList.append(cy)

            lmList.append([id, cx, cy])

        xmin, xmax = min(xList), max(xList)
        ymin, ymax = min(yList), max(yList)
        
        # bounding boxes
        bbox = xmin,ymin,xmax,ymax
        
        if draw :
            cv2.rectangle(img ,(bbox[0]-20,bbox[1]-20) ,(bbox[2]+20,bbox[3]+20), (0,255,0) ,2)
    return lmList , bbox

In [12]:
def fingersUp(lmList):
    
    fingers = []
    
    # Thumb
    if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
        fingers.append(1)
        
    else:
        fingers.append(0)
    
    # 4 Fingers
    for id in range(1, 5):
        if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
            fingers.append(1)
        else:
            fingers.append(0)
        
    return fingers

In [13]:
def findDistance(lmList,p1,p2,img,draw=True):
    x1,y1 = lmList[p1][1] , lmList[p1][2]
    x2,y2 = lmList[p2][1] , lmList[p2][2]


    cx,cy = (x1+x2)//2 ,(y1+y2)//2
    
    if draw:
        cv2.circle(img ,(x1,y1) ,5 ,(0,0,0) ,cv2.FILLED)
        cv2.circle(img ,(x2,y2) ,5 ,(0,0,0) ,cv2.FILLED)
        cv2.circle(img ,(cx,cy) ,5 ,(0,0,0) ,cv2.FILLED)

        cv2.line(img ,(x1,y1) ,(x2,y2) ,(230,85,73) ,3)

    length = math.hypot(x2-x1,y2-y1)
    
    return length,img ,[x1,y1,x2,y2,cx,cy]

In [14]:
# Frame reduction
frameR = 100

In [15]:
smoothening = 5

In [16]:
prev_loc_x , prev_loc_y = 0 , 0
curr_loc_x , curr_loc_y = 0 , 0

In [17]:
while True :
    _,img = cap.read()
    
    # Find Landmarks
    img,results = findHands(img)
    lmList ,bbox = findPosition(img ,results)
    
    # Get Tip of Index and Middle finger
    if len(lmList)!=0:
        # index finger
        x1,y1 = lmList[8][1:]
        # middle finger
        x2,y2 = lmList[12][1:]
        
        # Check which finger is up
        fingers = fingersUp(lmList)
        
        cv2.rectangle(img ,(frameR,frameR) ,(wCam-frameR,hCam-frameR) ,(255,0,255) ,2)
            
        # only index finger : Moving mode
        if fingers[1]==1 and fingers[2]==0 :
            # convert coordinates
            # converting from one range to another
            #           converting x1 , initial range ,
            x3 = np.interp(x1 ,(frameR,wCam-frameR) ,(0,wScrn))
            y3 = np.interp(y1 ,(frameR,hCam-frameR) ,(0,hScrn)) 
              
            # smoothen values , so it doesnot shake alot
            curr_loc_x = prev_loc_x + (x3-prev_loc_x)/smoothening
            curr_loc_y = prev_loc_y + (y3-prev_loc_y)/smoothening
           
            prev_loc_x , prev_loc_y = curr_loc_x , curr_loc_y

            # move mouse
            # we sub. with wScrn otherwise when we are moving to left move will move to right
            autopy.mouse.move(wScrn-curr_loc_x,curr_loc_y)
            cv2.circle(img ,(x1,y1) ,10 ,(0,0,0) ,-1)

        # Both index and middle finger are up : Clicking mode
        if fingers[1]==1 and fingers[2]==1 :
            # Find Distance between fingers
            length ,img ,line_info = findDistance(lmList,8,12,img)
            if length<50:
                cv2.circle(img ,(line_info[4],line_info[5]) ,10 ,(0,255,0) ,-1)
                # Click mouse if distance short
                autopy.mouse.click()
        
        

    cv2.imshow('AI Virtual Mouse',img)
    if cv2.waitKey(1)&0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()   